In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
main_dir = '../input/heart-attack-analysis-prediction-dataset/'
df = pd.read_csv(os.path.join(main_dir,'heart.csv'))
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
g = sns.countplot(x = 'output',data = df)

### Visualization based on Sex

In [ ]:
plt.figure(figsize=(18,10))
col = ['sex', 'cp', 'fbs', 'restecg','exng', 'slp', 'caa', 'thall',]
c = 1
for i in col:
    if c < 9:
        plt.subplot(2,4,c)
        sns.countplot(x = i,data =dataset,hue = 'output')
        plt.xlabel(i)
    c += 1
    plt.tight_layout()

### Converting Age from Numerical to Categorical 

In [ ]:
dataset = df.copy()
age_avg = dataset['age'].mean()
age_std = dataset['age'].std()
age_null_count = dataset['age'].isnull().sum()
    
age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
dataset['age'][np.isnan(dataset['age'])] = age_null_random_list
dataset['age'] = dataset['age'].astype(int)
    
dataset['CategoricalAge'] = pd.cut(dataset['age'], 5)

print (dataset[['CategoricalAge', 'output']].groupby(['CategoricalAge'], as_index=False).mean())

In [ ]:
dataset.loc[ dataset['age'] <= 29, 'age'] = 0
dataset.loc[(dataset['age'] > 29) & (dataset['age'] <= 39), 'age'] = 1
dataset.loc[(dataset['age'] > 39) & (dataset['age'] <= 48), 'age'] = 2
dataset.loc[(dataset['age'] > 48) & (dataset['age'] <= 58), 'age'] = 3
dataset.loc[(dataset['age'] > 58) & (dataset['age'] <= 67), 'age'] = 4
dataset.loc[ dataset['age'] > 77, 'age']= 5

dataset.head()

In [ ]:
dataset = dataset.drop(['CategoricalAge'],1)

### Age Range vs Risk

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(dataset.corr() ,annot = True)

In [ ]:
from numpy.random import seed
seed(0)
tf.random.set_seed(0)

In [ ]:
X = dataset.drop('output', axis=1)
y = dataset['output']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, RMSprop, SGD

In [ ]:
model = Sequential([
    Dense(32,activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation = 'sigmoid')
])

model.compile(optimizer = Adam(lr=0.001), loss = 'binary_crossentropy', metrics = ['acc'])

history = model.fit(X_train, y_train, batch_size = 32, epochs = 50)

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
legend = ax[0].legend(loc='best',shadow=True)
ax[1].plot(history.history['acc'], color='b', label="Training accuracy")
legend = ax[1].legend(loc='best',shadow=True)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
y_pred = model.predict(X_test)
y_pred  = y_pred > 0.5
cm = confusion_matrix(y_pred,y_test)
acc = accuracy_score(y_pred,y_test)

sns.heatmap(cm,annot = True)
plt.title('CONFUSION MATRIX')
print('Accuracy - {0:.2f}%'.format(acc*100))